In [ ]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

import numpy as np
import pandas as pd

options = Options()

#Comment this out if you want to see the browser
#options.add_argument("--headless")

pd.set_option('display.max_rows', 1000)

driver = webdriver.Chrome(options=options)

In [ ]:
def get_prop_lines(select, value):
    #Selects the prop passed as 'value' here from dropdown menu
    select.select_by_value(value)
    driver.implicitly_wait(10)

    #Initializes dataframe to store results
    result = pd.DataFrame(columns=["name", "team", "lines", "odds"])

    #fetches all games for a given prop and store them in 'games'
    games_cont = driver.find_element(By.CLASS_NAME, "wrapper")
    games = games_cont.find_elements(By.TAG_NAME, "app-game")

    for game in games:
        # Running into persistent proglem where the buttons for the individual games are not clicked properly so 
        # user has to click them when the program gets there (need to turn off headless mode to see this)
        # 4 lines below works for the MX games but not MLS for some reason
        driver.implicitly_wait(15)
        game.click()
        driver.implicitly_wait(2)
        game.click()

        game_content = WebDriverWait(game, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".content")))
        #Stores each individual prop bet in 'players'
        players = game_content.find_elements(By.CLASS_NAME, "player-group")
        print(f"Players: {len(players)}")
        for player in players:
            #Gets attributes for each individual prop bet and puts them all into one dataframe
            name = player.find_element(By.TAG_NAME, "b").text
            team = player.find_element(By.CLASS_NAME, "ss-team-names").text
            lines = player.find_element(By.TAG_NAME, "option").get_attribute("value")
            line = lines[0]
            odds = player.find_element(By.TAG_NAME, "button").text

            wager = pd.DataFrame([{"name": name, "team": team, "lines": line, "odds": odds}])
            frames = [result, wager]
            result = pd.concat(frames, ignore_index=True)

            print(name, team, line, odds)

        game.click()

    return result

In [ ]:
#Source function to run the scraper
def bovScrape_league_props(url):
    driver.get(url)
    driver.implicitly_wait(5)

    #Navigates to part of page where the prop bets are, enters the iframe and accesses the dropdown menu
    wait = WebDriverWait(driver, 10)
    league_box = wait.until(EC.presence_of_element_located((By.ID, "playerPropsLeagueFrame")))

    league_frame = league_box.find_element(By.TAG_NAME, "iframe")
    driver.switch_to.frame(league_frame)

    dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "select")))
    select = Select(dropdown)

    shots_lines = get_prop_lines(select, "Shots")
    shots_on_goal_lines = get_prop_lines(select, "Shots on Goal")


    return shots_lines, shots_on_goal_lines
    

In [ ]:
mex_url = 'https://www.bovada.lv/sports/soccer/north-america/mexico/liga-mx-apertura'
mls_url = 'https://www.bovada.lv/sports/soccer/north-america/united-states/mls'
shots, sogs = bovScrape_league_props(mls_url)
shots

In [ ]:
sogs